### Direct SDF Download from Excel CIDs

In [ ]:
import pandas as pd
import requests
import os
import time
import argparse

parser = argparse.ArgumentParser(description="Quick download of SDF files from clean CIDs")
parser.add_argument("--input", default="data/example_input.xlsx", help="Input Excel/CSV with PubChem IDs")
parser.add_argument("--output", default="outputs/sdf_files", help="Output folder for SDF files")
parser.add_argument("--cid_column", default="PubChem ID", help="Column name containing CIDs")
args = parser.parse_args()

if args.input.endswith(".xlsx"):
    df = pd.read_excel(args.input)
else:
    df = pd.read_csv(args.input)

raw_cids = df[args.cid_column].dropna().tolist()
clean_cids = [cid.replace("CID:", "").strip() for cid in raw_cids if str(cid).startswith("CID:") and cid.replace("CID:", "").strip().isdigit()]

os.makedirs(args.output, exist_ok=True)

for cid in clean_cids:
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF?record_type=3d"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            file_path = os.path.join(args.output, f"{cid}.sdf")
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"Downloaded CID {cid}")
        else:
            print(f"Failed CID {cid} (status {response.status_code})")
    except requests.exceptions.RequestException as e:
        print(f"Error for CID {cid}: {e}")

    time.sleep(0.2)
